In [5]:
import mnist
import numpy as np

In [3]:
x_train, t_train, x_test, t_test = mnist.load()

In [7]:
x_train=np.reshape(x_train,(60000,28,28,1))
x_test=np.reshape(x_test,(10000,28,28,1))

In [12]:
t_train[2]

4

In [24]:
import numpy as np

def conv(inputs, weights):
	# convolution layer with stride = 1
	# inputs[x,y,z], weights[n,x,y,z]
	i_size = inputs.shape
	w_size = weights.shape
	L = i_size[0] # inputs length
	D = i_size[2] # inputs/weights depth
	W = w_size[1] # weights length
	K = w_size[0] # number of weighs
	O_L = L-W+1 # outputs length
	O_D = K # outputs depth
	outputs = np.zeros((O_L, O_L, O_D))
	for n in range(0,K):
		for y_base in range(0,L-W+1):
			for x_base in range(0,L-W+1):
				for z in range(0,D):
					for y in range(y_base,y_base+W):
						for x in range(x_base,x_base+W):
							outputs[x_base,y_base,n] += inputs[x,y,z]*weights[n,x-x_base,y-y_base,z] 
	return outputs


def max_pool(inputs, pool_size):
	# Compute max-pooling on inputs (3-D matirx) using a window size of pool_size*pool_size
	# return the pooling results and posistion
	i_size = inputs.shape
	L = i_size[0] # inputs length
	D = i_size[2] # inputs depth
	O_L = int(L/pool_size)
	outputs = np.zeros((O_L, O_L, D))
	position = np.zeros((O_L, O_L, D))
	for z in range(0,D):
		for y in range(0,O_L):
			y_base = y*pool_size
			for x in range(0,O_L):
				x_base = x*pool_size
				max_num = -99999.99
				max_pos = 0
				for j in range(y_base,y_base+pool_size):
					for i in range(x_base,x_base+pool_size):
						if inputs[i,j,z]>max_num:
							max_num = inputs[i,j,z]
							max_pos = (i-x_base) + (j-y_base)*pool_size
				outputs[x,y,z] = max_num
				position[x,y,z] = max_pos
	return outputs, position


def relu(inputs):
	# Compute the relu function
	# inputs[x,y,z]
	return np.maximum(inputs,0)


def fc(inputs, weights):
	# inputs[x,y,z], weights[i,o]
	inputs_ar = inputs.flatten()
	inputs_nums = inputs_ar.shape[0]
	output_nums = weights.shape[1]
	outputs = np.zeros((output_nums,))
	for o in range(0,output_nums):
		for i in range(0,inputs_nums):
			outputs[o] += inputs_ar[i]*weights[i,o]
	return outputs

def softmax(x):
	# Compute softmax values for each number in inputs (1-D array)
	shift_x = x - np.max(x)
	exp_x = np.exp(shift_x)
	return exp_x / np.sum(exp_x)


def cross_entropy(outputs, label):
	# Compute the cross entropy loss
	# outputs is a 1-d array, label is a number (classification)
	return -np.log(outputs[label])
				
def back_conv(inputs, weights, out_gd):
	# Compute the gradients in one convolution layer
	# inputs[l,l,d], weights[k,w,w,d], out_gd[o_l,o_l,k]
	# The outputs are the weights gradients and inputs gradients
	i_size = inputs.shape
	w_size = weights.shape
	o_size = out_gd.shape
	L = i_size[0]
	D = i_size[2]
	W = w_size[1]
	K = w_size[0] # number of weighs and the depth of outputs
	O_L = o_size[0] # outputs length = L-W+1
	# Reshape out_gd[o_l,o_l,k] into [k,o_l,o_l,1]
	out_gd_rs = np.zeros((K,O_L,O_L,1))
	for n in range(0,K):
		for y in range(0,O_L):
			for x in range(0,O_L):
				out_gd_rs[n,x,y,0] = out_gd[x,y,n]
	w_gd = np.zeros(weights.shape)
	for d in range(0,D):
		tmp = conv(inputs[:,:,d].reshape((L,L,1)), out_gd_rs)
		for i in range(0,K):
			w_gd[i,:,:,d] = tmp[:,:,i]
	w_rs = np.zeros((D, W, W, K))
	for i in range(0,K):
		for j in range(0,D):
			w_rs[j,:,:,i] = weights[i,:,:,j] # w reshape
	w_rsrt = np.rot90(w_rs,2) # rotate 180
	pad_num = W-1
	out_gd_pad = np.pad(out_gd, ((pad_num,pad_num), (pad_num,pad_num), (0,0)), 'constant')
	i_gd = conv(out_gd_pad, w_rsrt)
	return w_gd, i_gd

def back_maxpool(out_gd, pos, pool_size):
	# Compute the gradients of max-pooling layer
	o_size = out_gd.shape
	i_size = (pool_size*o_size[0], pool_size*o_size[1], o_size[2])
	i_gd = np.zeros(i_size)
	for z in range(0,o_size[2]):
		for y in range(0,o_size[1]):
			y_base = y*pool_size
			for x in range(0,o_size[0]):
				x_base = x*pool_size
				x_add = int(pos[x,y,z]%pool_size)
				y_add = int(pos[x,y,z]/pool_size)
				i_gd[(x_base+x_add), (y_base+y_add), z] = out_gd[x,y,z]
	return i_gd

def back_relu(inputs, out_gd):
	# Compute the gradients of relu
	# inputs[l,l,d], out_gd[l,l,d]
	i_gd = out_gd
	i_size = inputs.shape
	for z in range(0,i_size[2]):
		for y in range(0,i_size[1]):
			for x in range(0,i_size[0]):
				if inputs[x,y,z]<=0:
					i_gd[x,y,z]=0
	return i_gd

def back_fc(inputs, weights, out_gd):
	# Compute the gradients in one FC layer
	# inputs[i_size], weights[i_size, o_size], out_gd[o_size]
	# The results are the weights gradients and inputs gradients
	i_size = inputs.shape[0]
	o_size = out_gd.shape[0]
	w_gd = np.zeros((i_size,o_size))
	for o in range(0,o_size):
		for i in range(0,i_size):
			w_gd[i,o] = out_gd[o]*inputs[i]
	i_gd = np.zeros((i_size,))
	for i in range(0,i_size):
		for o in range(0,o_size):
			i_gd[i] += out_gd[o]*weights[i,o]
	return w_gd, i_gd


def back_ce(outputs, label):
	# Compute the gradients of cross entropy loss on softmax inputs, d(a_k)=o_k-t_k
	# outputs is a 1-d array (softmax outputs), label is a number (classification)
	o_gd = outputs
	o_gd[label] -= 1
	return o_gd


In [17]:
nums = 10
inputs = np.random.rand(nums, 10, 10, 1)
w1_target = (np.random.rand(4, 3, 3, 1)-0.5)
w2_target = (np.random.rand(2, 3, 3, 4)-0.5)
w3_target = (np.random.rand(8, 4)-0.5)
# conv1: [10,10,1]*[4,3,3,1]=[8,8,4], relu
# pool2x2: [8,8,4]->[4,4,4]
# conv2: [4,4,4]*[2,3,3,4]=[2,2,2], relu
# fc: [2,2,2]->[4]
# softmax
def inference(inputs, w1, w2, w3):
	o1 = relu(conv(inputs, w1))
	x2, x2_pos = max_pool(o1, 2)
	o2 = relu(conv(x2, w2))
	o3 = fc(o2, w3)
	results = softmax(o3)
	return results
	# return results
def inference_train(inputs, w1, w2, w3):
	o1 = relu(conv(inputs, w1))
	x2, x2_pos = max_pool(o1, 2)
	o2 = relu(conv(x2, w2))
	o3 = fc(o2, w3)
	results = softmax(o3)
	return o1, x2_pos, o2, o3, results

In [18]:
# inference, generate data
outputs = np.zeros((nums,4))
for i in range(0,nums):
 	outputs[i,:] = inference(inputs[i,:,:,:], w1_target, w2_target, w3_target)
labels = np.argmax(outputs, axis=1)

In [19]:
w1 = (np.random.rand(4, 3, 3, 1)-0.5)
w2 = (np.random.rand(2, 3, 3, 4)-0.5)
w3 = (np.random.rand(8, 4)-0.5)

w1_gd = np.zeros((4, 3, 3, 1))
w2_gd = np.zeros((2, 3, 3, 4))
w3_gd = np.zeros((8, 4))

# train, SGD
lr = 0.001
for i in range(0,nums):
	x = np.random.randint(nums, size=1)[0]
	o1, x2_pos, o2, o3, o = inference_train(inputs[x,:,:,:], w1, w2, w3)
	label = labels[x]
	o_gd = back_ce(o, label)
	w3_gd, i3_gd = back_fc(o2.flatten(), w3, o_gd)
	w3 -= lr*w3_gd
	o2_gd = np.reshape(i3_gd, (2,2,2))
	w2_gd, i2_gd = back_conv(x2, w2, back_relu(o2, o2_gd))
	w2 -= lr*w2_gd
	w1_gd, i1_gd = back_conv(inputs[x,:,:,:], w1, back_relu(o1 ,back_maxpool(i2_gd, x2_pos, 2)))
	w1 -= lr*w1_gd

NameError: name 'x2' is not defined

In [10]:
0

0

In [9]:
c1_size = (8, 5, 5, 1)
c2_size = (16, 5, 5, 8)
f1_size = (256, 256)
f2_size = (256, 10)

In [35]:
def mnist_in_only(inputs, w1, w2, w3, w4):
	o1 = relu(conv(inputs, w1))
	x2, x2_pos = max_pool(o1, 2)
	o2 = relu(conv(x2, w2))
	x3, x3_pos = max_pool(o2, 2)
	o3 = fc(x3, w3)
	o4 = fc(o3, w4)
	results = softmax(o4)
	return np.argmax(results)

def mnist_in(inputs, w1, w2, w3, w4):
	o1 = relu(conv(inputs, w1))
	x2, x2_pos = max_pool(o1, 2)
	o2 = relu(conv(x2, w2))
	x3, x3_pos = max_pool(o2, 2)
	o3 = fc(x3, w3)
	o4 = fc(o3, w4)
	results = softmax(o4)
	return o1, x2_pos, x2, o2, x3_pos, x3, o3, results

def evaluate_train(n, w1, w2, w3, w4):
	hit = 0
	for i in range(0,n):
		x = np.random.randint(60000, size=1)[0]
		y = mnist_in_only(x_train[x,:,:,:], w1, w2, w3, w4)
		label = t_train[x]
		if label == y:
			hit += 1
	return hit/n

In [ ]:
c1 = (np.random.rand(c1_size[0],c1_size[1],c1_size[2],c1_size[3])-0.5)
c2 = (np.random.rand(c2_size[0],c2_size[1],c2_size[2],c2_size[3])-0.5)
f1 = (np.random.rand(f1_size[0], f1_size[1])-0.5)
f2 = (np.random.rand(f2_size[0], f2_size[1])-0.5)

In [ ]:
nums = 10000

# train, SGD
lr = 0.001
for i in range(0,nums):
	x = np.random.randint(60000, size=1)[0]
	o1, x2_pos, x2, o2, x3_pos, x3, o3, o = mnist_in(x_train[x,:,:,:], c1, c2, f1, f2)
	label = t_train[x]
	o_gd = back_ce(o, label)
	f2_gd, i4_gd = back_fc(o3.flatten(), f2, o_gd)
	f1_gd, i3_gd = back_fc(x3.flatten(), f1, i4_gd)
	
	o2_gdrs = np.reshape(i3_gd, (4,4,16))
	o2_gd = back_maxpool(o2_gdrs, x3_pos, 2)
	c2_gd, i2_gd = back_conv(x2, c2, back_relu(o2, o2_gd))

	o1_gd = back_maxpool(i2_gd, x2_pos, 2)
	c1_gd, i1_gd = back_conv(x_train[x,:,:,:], c1, back_relu(o1, o1_gd))
	
	f2 -= lr*f2_gd
	f1 -= lr*f1_gd
	c2 -= lr*c2_gd
	c1 -= lr*c1_gd
	if i%500==0:
		print(evaluate_train(50, c1, c2, f1, f2))

0.1
0.04
0.12
0.14
0.08
0.08
0.14
0.08
0.08
0.16
0.06


In [33]:
np.argmax(mnist_in_only(x_train[100,:,:,:], c1, c2, f1, f2))

8

In [29]:
t_train[0]

5

In [38]:
print(evaluate_train(1000, c1, c2, f1, f2))

0.087


[[0.79417445 0.17329868]
 [0.86185539 0.93331098]
 [0.10208973 0.58916474]
 [0.99718015 0.83547482]]
[[0.8232517  0.50407707]
 [0.7168652  0.56350755]
 [0.88392545 0.38008863]
 [0.62029238 0.72771073]]
[[0.72552191 0.03438626]
 [0.54815919 0.45128632]
 [0.55896284 0.84450879]
 [0.81781409 0.52432925]]
[[0.93402148 0.62054184]
 [0.9162403  0.66802162]
 [0.91838777 0.1487008 ]
 [0.22275874 0.61763001]]
